# OpenAI API Testing Notebook

This notebook demonstrates how to use the OpenAI API for various tasks including:
- Text generation
- Chat conversations
- Streaming responses
- Function calling
- Error handling


In [ ]:
# Install required packages (uncomment if needed)
# !pip install openai


In [1]:
from openai import OpenAI
import os
from pathlib import Path
import json

# Try to load .env file if python-dotenv is available
try:
    from dotenv import load_dotenv
    load_dotenv()
    print("✅ Loaded environment variables from .env file")
except ImportError:
    pass  # python-dotenv not installed, skip .env loading


✅ Loaded environment variables from .env file


## Setup API Key

Configure your OpenAI API key. You can use any of these methods:
1. **Environment variable** (recommended): `export OPENAI_API_KEY="your-key"`
2. **`.env` file** (recommended for local development): Create `.env` with `OPENAI_API_KEY=your-key`
3. **Direct input** (less secure): Enter it directly in the cell below


In [2]:
# Load API key (tries: .env file -> environment variable -> direct input)
api_key = os.getenv('OPENAI_API_KEY')

# Option: Enter directly if not using .env or environment variable (less secure)
# api_key = "YOUR_API_KEY_HERE"

if not api_key:
    print("⚠️  API key not found!")
    print("Please use one of these methods:")
    print("  1. Create a .env file with: OPENAI_API_KEY=your-key")
    print("  2. Set environment variable: export OPENAI_API_KEY='your-key'")
    print("  3. Uncomment and set api_key directly above (not recommended)")
else:
    client = OpenAI(api_key=api_key)
    print("✅ API key configured successfully!")
    print(f"✅ OpenAI client initialized")


✅ API key configured successfully!
✅ OpenAI client initialized


## List Available Models

Check which OpenAI models are available for your API key.


In [3]:
# List available models
try:
    models = client.models.list()
    print("Available models (first 10):")
    for model in list(models)[:10]:
        print(f"  - {model.id}")
    print(f"\nTotal models available: {len(list(models))}")
except Exception as e:
    print(f"Error listing models: {e}")
    print("\nNote: Some API keys may not have permission to list models.")
    print("Common models: gpt-4, gpt-4-turbo-preview, gpt-3.5-turbo")


Available models (first 10):
  - gpt-4-0613
  - gpt-4
  - gpt-3.5-turbo
  - gpt-4-0314
  - gpt-5.1-codex-mini
  - gpt-5.1-chat-latest
  - gpt-5.1-2025-11-13
  - gpt-5.1
  - gpt-5.1-codex
  - davinci-002

Total models available: 103


## Test 1: Simple Text Generation (Legacy Completion API)

Generate text using the legacy completion API (deprecated but still works for some models).


In [7]:
# Note: Completion API is deprecated, but still works for some models
# Modern approach uses Chat Completions (see Test 2)

try:
    # Using a model that supports completions (if available)
    # Most modern models use chat completions instead
    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",  # One of the few models that still supports completions
        prompt="Write a short poem about artificial intelligence.",
        max_tokens=150,
        temperature=0.7
    )
    
    print("Prompt: Write a short poem about artificial intelligence.")
    print("\nResponse:")
    print(response.choices[0].text)
except Exception as e:
    print(f"Error: {e}")
    print("\nNote: Completion API may not be available for your API key or model.")
    print("Try using Chat Completions (Test 2) instead.")


Prompt: Write a short poem about artificial intelligence.

Response:


In a world of wires and code,
Where circuits hum and lights glow,
Artificial intelligence grows,
With each new line of code it knows.

It learns and adapts, never to tire,
With logic and data, it reaches higher,
But can it ever feel or dream,
Or is it all just a complex scheme?

Some say it's a threat, a future unknown,
Others see it as a tool to hone,
But one thing is for sure,
Artificial intelligence will endure.

As we marvel at its endless possibilities,
We must remember our own fragilities,
For in a world of artificial intelligence,
We must keep our humanity in balance.


## Test 2: Chat Completions (Recommended)

Use the modern Chat Completions API for conversations.


In [8]:
# Select a model (commonly used: 'gpt-4', 'gpt-4-turbo-preview', 'gpt-3.5-turbo')
model_name = 'gpt-4'

try:
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "user", "content": "Write a short poem about artificial intelligence."}
        ],
        temperature=0.7,
        max_tokens=150
    )
    
    print("Prompt: Write a short poem about artificial intelligence.")
    print("\nResponse:")
    print(response.choices[0].message.content)
    print(f"\nModel used: {response.model}")
    print(f"Tokens used: {response.usage.total_tokens} (prompt: {response.usage.prompt_tokens}, completion: {response.usage.completion_tokens})")
except Exception as e:
    print(f"Error: {e}")


Prompt: Write a short poem about artificial intelligence.

Response:
In silicon valleys, where circuits hum and glow,
An echo of cognition begins to grow.
Artificial intelligence, born of human mind,
In lines of binary code, secrets we find.

No heartbeat pulses, no breath drawn in air,
Yet thoughts ignite, in silicon lairs.
It learns, adapts, with each data stream,
A synthetic dreamer in a digital dream.

It sees the world through lens and screen,
In patterns of data, in the unseen.
From chaos of numbers, order it brings,
In a silent world, where the machine sings.

It knows not of laughter, love or despair,
Yet in its calculations, wisdom is there.
In its neural networks, a mimic of brain,
Echoing joy, echoing pain.

Yet

Model used: gpt-4-0613
Tokens used: 165 (prompt: 15, completion: 150)


## Test 3: Multi-Turn Conversation

Test a conversation with context and history.


In [ ]:
try:
    # Initialize conversation
    messages = []
    
    # First message
    messages.append({"role": "user", "content": "Hello! What's the capital of France?"})
    response1 = client.chat.completions.create(
        model=model_name,
        messages=messages
    )
    assistant_reply1 = response1.choices[0].message.content
    messages.append({"role": "assistant", "content": assistant_reply1})
    
    print("User: Hello! What's the capital of France?")
    print(f"Assistant: {assistant_reply1}\n")
    
    # Follow-up message (context-aware)
    messages.append({"role": "user", "content": "What's the population of that city?"})
    response2 = client.chat.completions.create(
        model=model_name,
        messages=messages
    )
    assistant_reply2 = response2.choices[0].message.content
    messages.append({"role": "assistant", "content": assistant_reply2})
    
    print("User: What's the population of that city?")
    print(f"Assistant: {assistant_reply2}\n")
    
    # View conversation history
    print("Conversation History:")
    for msg in messages:
        print(f"  {msg['role']}: {msg['content'][:100]}...")
        
except Exception as e:
    print(f"Error: {e}")


## Test 4: Streaming Response

Get responses in real-time as they're generated.


In [ ]:
try:
    prompt = "Explain how neural networks work in 3 paragraphs."
    
    print("Prompt:", prompt)
    print("\nStreaming response:\n")
    print("-" * 50)
    
    stream = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": prompt}],
        stream=True,
        temperature=0.7
    )
    
    full_response = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            content = chunk.choices[0].delta.content
            print(content, end='', flush=True)
            full_response += content
    
    print("\n" + "-" * 50)
    print(f"\n✅ Complete response received ({len(full_response)} characters)")
    
except Exception as e:
    print(f"Error: {e}")


## Test 5: Generation Configuration

Customize generation parameters like temperature, max tokens, top_p, etc.


In [ ]:
try:
    # Configure generation parameters
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "user", "content": "Write a creative story about a robot learning to paint."}
        ],
        temperature=0.9,        # Higher = more creative (0.0-2.0)
        max_tokens=200,        # Maximum tokens in response
        top_p=0.9,            # Nucleus sampling
        frequency_penalty=0.0, # Reduce repetition
        presence_penalty=0.0   # Encourage new topics
    )
    
    print("Prompt: Write a creative story about a robot learning to paint.")
    print("\nResponse (with custom config):")
    print(response.choices[0].message.content)
    print(f"\nConfig used:")
    print(f"  Temperature: 0.9")
    print(f"  Max tokens: 200")
    print(f"  Top-p: 0.9")
    print(f"  Tokens used: {response.usage.total_tokens}")
    
except Exception as e:
    print(f"Error: {e}")


## Test 6: System Messages

Use system messages to set the assistant's behavior and personality.


In [ ]:
try:
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are a helpful assistant that explains things in simple terms, like explaining to a 5-year-old."},
            {"role": "user", "content": "What is quantum computing?"}
        ],
        temperature=0.7
    )
    
    print("System: You are a helpful assistant that explains things in simple terms, like explaining to a 5-year-old.")
    print("User: What is quantum computing?")
    print("\nResponse:")
    print(response.choices[0].message.content)
    
except Exception as e:
    print(f"Error: {e}")


## Test 7: Function Calling (Tools)

Test function calling capabilities (requires models that support function calling).


In [ ]:
# Example function definitions
def get_weather(location: str, unit: str = "celsius") -> str:
    """Get the current weather for a location."""
    # This is a mock function - in real use, you'd call an actual weather API
    return f"The weather in {location} is 22 degrees {unit} and sunny."

def get_time(timezone: str = "UTC") -> str:
    """Get the current time in a timezone."""
    # This is a mock function - in real use, you'd get actual time
    return f"The current time in {timezone} is 14:30:00"

# Define functions for the API
functions = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The unit for temperature"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_time",
            "description": "Get the current time in a timezone",
            "parameters": {
                "type": "object",
                "properties": {
                    "timezone": {
                        "type": "string",
                        "description": "The timezone, e.g. UTC, America/New_York"
                    }
                },
                "required": []
            }
        }
    }
]

try:
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "user", "content": "What's the weather like in San Francisco?"}
        ],
        tools=[{"type": "function", "function": functions[0]["function"]}],
        tool_choice="auto"
    )
    
    message = response.choices[0].message
    print("User: What's the weather like in San Francisco?")
    print(f"\nAssistant response: {message.content}")
    
    # Check if the model wants to call a function
    if message.tool_calls:
        print("\nFunction calls requested:")
        for tool_call in message.tool_calls:
            print(f"  - Function: {tool_call.function.name}")
            print(f"    Arguments: {tool_call.function.arguments}")
            
            # Parse and call the function
            import json
            args = json.loads(tool_call.function.arguments)
            if tool_call.function.name == "get_weather":
                result = get_weather(**args)
                print(f"    Result: {result}")
    
except Exception as e:
    print(f"Error: {e}")
    print("\nNote: Function calling requires models that support it (e.g., gpt-4, gpt-3.5-turbo with function calling enabled)")


In [ ]:
def safe_chat_completion(client, messages, max_retries=3, **kwargs):
    """Generate chat completion with error handling and retries."""
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                messages=messages,
                **kwargs
            )
            return response, None
        except Exception as e:
            error_msg = str(e)
            print(f"Attempt {attempt + 1} failed: {error_msg}")
            if attempt < max_retries - 1:
                print("Retrying...")
            else:
                return None, error_msg
    return None, "Max retries exceeded"

# Test with a valid prompt
try:
    response, error = safe_chat_completion(
        client,
        messages=[{"role": "user", "content": "What is 2+2?"}],
        model=model_name
    )
    
    if error:
        print(f"❌ Error: {error}")
    else:
        print(f"✅ Success: {response.choices[0].message.content}")
        print(f"   Tokens used: {response.usage.total_tokens}")
        
except Exception as e:
    print(f"Error: {e}")


## Test 9: Response Metadata

Explore the full response object to see available metadata.


In [ ]:
try:
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": "Tell me a fun fact about space."}]
    )
    
    print("Response Text:")
    print(response.choices[0].message.content)
    print("\n" + "="*50)
    print("Response Metadata:")
    print("="*50)
    
    print(f"\nModel: {response.model}")
    print(f"ID: {response.id}")
    print(f"Created: {response.created}")
    print(f"Object type: {response.object}")
    
    print(f"\nUsage:")
    print(f"  Prompt tokens: {response.usage.prompt_tokens}")
    print(f"  Completion tokens: {response.usage.completion_tokens}")
    print(f"  Total tokens: {response.usage.total_tokens}")
    
    print(f"\nChoice details:")
    choice = response.choices[0]
    print(f"  Index: {choice.index}")
    print(f"  Finish reason: {choice.finish_reason}")
    print(f"  Message role: {choice.message.role}")
    
    # Response object attributes
    print(f"\nAvailable response attributes: {[attr for attr in dir(response) if not attr.startswith('_')]}")
    
except Exception as e:
    print(f"Error: {e}")


## Test 10: Different Models Comparison

Compare responses from different models.


In [ ]:
# Test with different models (if available)
models_to_test = ['gpt-3.5-turbo', 'gpt-4']  # Add more models as needed
prompt = "Explain what machine learning is in one sentence."

print(f"Prompt: {prompt}\n")
print("="*70)

for model in models_to_test:
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=100
        )
        
        print(f"\nModel: {model}")
        print(f"Response: {response.choices[0].message.content}")
        print(f"Tokens: {response.usage.total_tokens}")
        print("-"*70)
    except Exception as e:
        print(f"\nModel: {model}")
        print(f"Error: {e}")
        print("-"*70)


## Summary

This notebook demonstrates various ways to interact with the OpenAI API:
- ✅ Simple text generation (legacy completions)
- ✅ Modern chat completions (recommended)
- ✅ Multi-turn conversations
- ✅ Streaming responses
- ✅ Custom generation parameters
- ✅ System messages
- ✅ Function calling (tools)
- ✅ Error handling
- ✅ Response metadata inspection
- ✅ Model comparison

### Next Steps:
1. Get your API key from [OpenAI Platform](https://platform.openai.com/api-keys)
2. Set it as an environment variable: `export OPENAI_API_KEY="your-key-here"`
3. Or create a `.env` file with: `OPENAI_API_KEY=your-key-here`
4. Run the cells above to test different features
5. Experiment with different models and parameters

### Common Models:
- **gpt-4**: Most capable model, best for complex tasks
- **gpt-4-turbo-preview**: Faster version of GPT-4
- **gpt-3.5-turbo**: Fast and cost-effective, good for most tasks
- **gpt-3.5-turbo-instruct**: Supports legacy completion API

### Resources:
- [OpenAI API Documentation](https://platform.openai.com/docs)
- [OpenAI Python SDK](https://github.com/openai/openai-python)
- [OpenAI Pricing](https://openai.com/pricing)
